In [2]:
from transformers import AutoModel
import torch


model = AutoModel.from_pretrained("huggyllama/llama-7b", trust_remote_code=True, load_in_8bit=True, device_map='auto')

In [8]:
from modelscope import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [9]:
from peft import PeftModel

model = PeftModel.from_pretrained(model, "./output/")

/home/mymusise/pro/ChatGLM-Tuning/peft/tuners/lora.py:175: UserWarning: fan_in_fan_out is set to True but the target module is not a Conv1D. Setting fan_in_fan_out to False.
  warnings.warn(


In [10]:
import json

instructions = json.load(open("data/alpaca_data.json"))

In [11]:
answers = []
from cover_alpaca2jsonl import format_example


with torch.no_grad():
    for idx, item in enumerate(instructions[:3]):
        feature = format_example(item)
        input_text = feature['context']
        ids = tokenizer.encode(input_text)
        input_ids = torch.LongTensor([ids])
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            do_sample=False,
            temperature=0
        )
        out_text = tokenizer.decode(out[0])
        answer = out_text.replace(input_text, "").replace("\nEND", "").strip()
        item['infer_answer'] = answer
        print(out_text)
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')
        answers.append({'index': idx, **item})

/home/mymusise/pro/ChatGLM-Tuning/transformers/generation/utils.py:1374: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Instruction: Give three tips for staying healthy.
Answer: 1. Eat a balanced diet.
2. Get regular exercise.
3. Stay hydrated.
### 1.Answer:
 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule. 


Instruction: What are the three primary colors?
Answer: The three primary colors are red, blue, and yellow.
### 2.Answer:
 The three primary colors are red, blue, and yellow. 


Instruction: Describe the structure of an atom.
Answer: An atom is a small particle of an element, containing a small number of particles of the element. Each particle is made up of a single electron, which is surrounded by a cloud of negative charge. The cloud of negative charge is surrounded by a cloud of positive charge, which creates a neutral cloud. The neutral cloud is made up of a cloud of negative charge, which creates a cloud of positive charge. This process cont